In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
from math import sqrt
import matplotlib.colors as mcol
import matplotlib.cm as cm
import matplotlib as mpl


%matplotlib notebook

np.random.seed(12345) #To ensure the same random numbers are generated everytime

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[2016,2017,2018,2019])
df

ImportError: Cannot load backend 'Qt4Agg' which requires the 'qt4' interactive framework, as 'qt5' is currently running

#### *Calculating the mean for the samples and the standard deviation*

In [10]:
mean = [np.mean(df.loc[i]) for i in df.index]
std = [np.std(df.loc[i]) for i in df.index]

fig,ax = plt.subplots()
ax.bar(df.index,mean,alpha=0.5)
fig.set_size_inches(8,5)
ax.set_xticks(df.index)
ax.set_xlabel('Years')
ax.set_ylabel('Mean fuel consumed in Metric Tonnes (MT)')
ax.set_title('Xanardia Fuel Usage Trends')

Text(0.5, 1.0, 'Xanardia Fuel Usage Trends')

#### *Calculating the margin of error for the confidence interval of mean* 
* P.S. This is different from the confidence interval for the normal distribution
* Here 1.96 is the z value for confidence level = 0.95 for a normal distribution

In [4]:
margin_of_error = np.array([1.96*std[i]/sqrt(len(df.iloc[i])) for i in range(0,len(df.index))])
margin_of_error

fig,ax = plt.subplots()
bar = ax.bar(df.index,mean,alpha=0.5, yerr=margin_of_error, capsize=10)
fig.set_size_inches(8,5)
ax.set_xticks(df.index)
ax.set_xlabel('Years')
ax.set_ylabel('Mean fuel consumed in Metric Tonnes (MT)')
ax.set_title('Xanardia Fuel Usage Trends with 95% confidence interval')

# for rect in bar:
#     height = rect.get_height()
#     plt.text(rect.get_x() + rect.get_width()/2,height/2, '%d' % int(height), ha='center', va='bottom')


Text(0.5, 1.0, 'Xanardia Fuel Usage Trends with 95% confidence interval')

#### *Creating a colormap*

In [5]:
cmap = plt.cm.seismic  # define the colormap

# extract all colors from the .seismic map
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

#cm1 = mcol.LinearSegmentedColormap.from_list("Test",["b", "white", "purple"])

cpick = cm.ScalarMappable(cmap=cmap) 
cpick.set_array([])


# define the bins and normalize
bounds = np.linspace(0, 1, 12)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [6]:
#bars[1].set_color(cmaplist[0])

In [7]:
x = [i for i in df.index]
fig,ax = plt.subplots()
#plt.errorbar(df.index,mean,linestyle='')
fig.set_size_inches(9,6)
fig.subplots_adjust(right=0.75)

bars = ax.bar(df.index,mean,yerr=margin_of_error,capsize=10,bottom=0,alpha=0.8,edgecolor='black')
ax.set_xticks(np.arange(min(x),max(x)+1,1))
ax.set_xlabel('Years')
ax.set_ylabel('Mean fuel consumed in Metric Tonnes (MT)')

start_ypoint = 20000
hline = ax.axhline(start_ypoint,c='grey')
line_text = ax.text(1.01, start_ypoint, "T={}".format(start_ypoint), va='center', ha="left", bbox=dict(facecolor="w",alpha=0.5),
                    transform=ax.get_yaxis_transform())


# create a second axes for the colorbar
ax2 = fig.add_axes([0.85, 0.1, 0.03, 0.8])

cbar = fig.colorbar(cpick, cmap=cmap, norm=norm, spacing='proportional', ticks=bounds,
             boundaries=bounds, format='%1.2f',alpha=0.8, orientation='vertical',cax=ax2)


cbar.ax.set_ylabel('P(Mean > T)')

ax.set_title('Xanardia Fuel Usage Trends with 95% confidence interval')

Text(0.5, 1.0, 'Xanardia Fuel Usage Trends with 95% confidence interval')

#### Upon User-Click setting the Y coordinates

In [8]:
def probability_mean_above_threshold(threshold):
    probabilities=[]
    for i,bar in enumerate(bars):
        
        if threshold>=bar.get_height()+margin_of_error[i]: 
            prob=0
        
        elif threshold<=bar.get_height()-margin_of_error[i]:
            prob=1
        
        else:   
            prob = ( (bar.get_height()+margin_of_error[i]) - threshold)/(2*margin_of_error[i])
        
        probabilities.append(prob)
    
    return probabilities


def update_bars(threshold):
        prob = probability_mean_above_threshold(threshold)
        
        for bar, p in zip(bars, prob):
            #bar.set_color(cb.get_clim)
            bar.set_color(cpick.to_rgba(p))
                

def on_click(event):
    if event.inaxes == ax:
        hline.set_ydata(event.ydata)
        line_text.set_position((1.01,event.ydata))
        line_text.set_text("T={:.0f}".format(event.ydata))        
        #print(prob)
        update_bars(event.ydata)
        fig.canvas.draw_idle()
        

cid = fig.canvas.mpl_connect('button_press_event', on_click)

update_bars(start_ypoint)